# import essential librarys

In [95]:
import pandas as pd
import numpy as np
from pyomo.environ import *

In [93]:
# Load Profile
df_load = pd.read_csv('load_profile.csv', index_col=0, header=0)

# Sets
buses = [1, 2, 3]
generators = ["g1", "g2", "g3"]
time_periods = range(1, 25)

# Parameters
Sbase = 100

# Generator Data
gen_data = {
    "g1": {"b": 50, "pmin": 0, "pmax": 350},
    "g2": {"b": 150, "pmin": 0, "pmax": 350},
    "g3": {"b": 100, "pmin": 0, "pmax": 350},
}

# Generator-Bus Connectivity
gen_connect = {"g1": 1, "g2": 2, "g3": 3}

# Bus Data (Demands in MW)
bus_data = {2: 480}

# Branch Data
branch_data = {
    (1, 2): {"x": 0.01, "limit": 400},
    (2, 3): {"x": 0.01, "limit": 240},
    (1, 3): {"x": 0.01, "limit": 160},
}
for (i, j), v in list(branch_data.items()):
    branch_data[j, i] = v  

# Model
model = ConcreteModel()

# Variables
model.Pg = Var(generators, time_periods, within=NonNegativeReals)
model.Pij = Var(branch_data.keys(), time_periods, within=Reals)
model.delta = Var(buses, time_periods, bounds=(-np.pi, np.pi))

# Objective Function
def obj_rule(m):
    return sum(m.Pg[g, t] * gen_data[g]["b"] * Sbase for g in generators for t in time_periods)
model.OF = Objective(rule=obj_rule, sense=minimize)

# Power Flow Constraints
def power_flow_rule(m, i, j, t):
    if (i, j) in branch_data:
        return m.Pij[i, j, t] == (1 / branch_data[i, j]["x"]) * (m.delta[i, t] - m.delta[j, t])
    return Constraint.Skip
model.const1 = Constraint(branch_data.keys(), time_periods, rule=power_flow_rule)

# Power Balance Constraints
def power_balance_rule(m, b, t):
    gen_term = sum(m.Pg[g, t] for g in generators if gen_connect[g] == b)
    load_term = bus_data.get(b, 0) * df_load.iloc[t-1, 0] / Sbase
    flow_term = sum(m.Pij[b, j, t] for j in buses if (b, j) in branch_data)
    return gen_term - load_term == flow_term
model.const2 = Constraint(buses, time_periods, rule=power_balance_rule)

# Ramp Up & Down Constraints
def ramp_up_rule(m, g, t):
    if t < 24:  
        return m.Pg[g, t+1] - m.Pg[g, t] <= 0.2 * gen_data[g]["pmax"] / Sbase
    else:
        return Constraint.Skip  # Skip for the last time period
model.ramp_up = Constraint(generators, time_periods, rule=ramp_up_rule)

def ramp_down_rule(m, g, t):
    if t < 24:  
        return m.Pg[g, t] - m.Pg[g, t+1] <= 0.2 * gen_data[g]["pmax"] / Sbase
    else:
        return Constraint.Skip  # Skip for the last time period
model.ramp_down = Constraint(generators, time_periods, rule=ramp_down_rule)


# Generator Limits
for g in generators:
    for t in time_periods:
        model.Pg[g, t].setlb(gen_data[g]["pmin"] / Sbase)
        model.Pg[g, t].setub(gen_data[g]["pmax"] / Sbase)

# Slack Bus
for t in time_periods:
    model.delta[1, t].fix(0)

# Branch Limits
for (i, j) in branch_data.keys():
    for t in time_periods:
        model.Pij[i, j, t].setlb(-branch_data[i, j]["limit"] / Sbase)
        model.Pij[i, j, t].setub(branch_data[i, j]["limit"] / Sbase)

        
# dual values
model.dual = Suffix(direction=Suffix.IMPORT)

# Solve
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True, keepfiles=True, symbolic_solver_labels=True)

# Results
report = {}
for t in time_periods:
    report[t] = {
        g: value(model.Pg[g, t]) * Sbase for g in generators
    }
    report[t].update({
        f"delta{b}": value(model.delta[b, t]) for b in buses
    })
    report[t].update({
        f"LMP{b}": model.dual[model.const2[b, t]] / Sbase for b in buses
    })
    
    report[t].update({
        f"Load{b}": bus_data.get(b, 0) * df_load.iloc[t-1, 0] for b in buses
    })
    

# Print Results
for t, data in report.items():
    print("\nHour", t, "=" * 50)
    print(data)
    for (i, j) in branch_data.keys():
        print(f"LineFlow_{i}_{j} = ", value(model.Pij[i, j, t]) * Sbase)

Solver log file: 'C:\Users\pedis\AppData\Local\Temp\tmpenfr4_qn.glpk.log'
Solver solution file: 'C:\Users\pedis\AppData\Local\Temp\tmp9o7cxfpm.glpk.raw'
Solver problem files: ('C:\\Users\\pedis\\AppData\\Local\\Temp\\tmp_alhqen7.pyomo.lp',)
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\pedis\AppData\Local\Temp\tmp9o7cxfpm.glpk.raw --wglp C:\Users\pedis\AppData\Local\Temp\tmpmrhc_sqx.glpk.glp
 --cpxlp C:\Users\pedis\AppData\Local\Temp\tmp_alhqen7.pyomo.lp
Reading problem data from 'C:\Users\pedis\AppData\Local\Temp\tmp_alhqen7.pyomo.lp'...
355 rows, 265 columns, 829 non-zeros
2238 lines were read
Writing problem data to 'C:\Users\pedis\AppData\Local\Temp\tmpmrhc_sqx.glpk.glp'...
1951 lines were written
GLPK Simplex Optimizer 5.0
355 rows, 265 columns, 829 non-zeros
Preprocessing...
354 rows, 264 columns, 828 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+02  ratio =  1.000e+02
GM: min|aij| =  1.000e+00  max|aij| =  1.000